In [1]:
import csv, urllib2, json, datetime, subprocess
import dateutil.parser as dp
from dateutil import rrule

In [2]:
awairRoot = 'https://beta-api.awair.is/v1/'

In [3]:
def exec_ipynb(url):
    import json, re, urllib2
    nb = (urllib2.urlopen(url) if re.match(r'https?:', url) else open(url)).read()
    #had to change this statement for compatibility with the latest version of Jupyter Notebook
    exec '\n'.join([''.join(cell['source']) for cell in json.loads(nb)['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('python-utils/esdr-library.ipynb')

In [4]:
# First time uploading, create a new client like so:

# Esdr.save_client('esdr-auth-awair-uploader.json', 'awair uploader for timemachine1')

# and then follow the directions it prints, which include visiting esdr.cmucreatelab.org and creating
# a client with given parameters, and also editing esdr-auth-baaqm-uploader.json to include your
# username and password

# Do not add esdr-auth-baaqm-uploader.json to the git repo

In [5]:
esdr = Esdr('esdr-auth-awair-uploader.json')
product = esdr.get_or_create_product('awair', 'awair', 'Sensor made by Awair')
product

{u'created': u'2016-05-17T22:08:14.000Z',
 u'creatorUserId': 1,
 u'defaultChannelSpecs': {u'channels': {u'pm': {u'prettyName': u'Particle Concentration',
    u'range': {u'max': None, u'min': 0},
    u'units': u'ug/m^3'}},
  u'version': 1},
 u'description': u'The Awair air quality monitor',
 u'id': 58,
 u'modified': u'2016-05-18T13:14:20.000Z',
 u'name': u'awair',
 u'prettyName': u'Awair',
 u'vendor': u'Bitfinder'}

In [6]:
#parse awair-sensors-metadata.csv for unique API keys
def parseForApiKeys(file): 
    apiKeys = set([])
    with open(file, 'rb') as csvfile:
        reader = csv.reader(csvfile)
        reader.next() #skip the header row
        for row in reader:
            key = row[6]
            if key:
                apiKeys.update([key])
    return apiKeys
#parseForApiKeys('awair-sensors-metadata.csv')

In [7]:
def parseForDevicesAndStartDates(file):
    devices = {}
    with open(file, 'rb') as csvfile:
        reader = csv.reader(csvfile)
        reader.next() #skip the header row
        for row in reader:
            deviceId = row[5]
            if deviceId:
                devices[deviceId] = row[1]
    return devices

In [8]:
def getDevicesForAwairUser(apiKey):
    url = '%susers/self/devices' % awairRoot
    header = 'Authorization: Bearer %s' % apiKey
    #using curl because Awair's API requires the latest SSL
    output = subprocess.check_output(["curl", "-q", url,"-H",header])
    devices = json.loads(output)['data']
    return devices

In [9]:
def getEsdrFeeds(apiKey):
    feeds = {}
    devices = getDevicesForAwairUser(apiKey)
    for device in devices:
        #create an ESDR device and feed for each
        awairDeviceId = str(device['device_id'])
        esdrDevice = esdr.get_or_create_device(product, awairDeviceId, device['device_name'])
        feed = esdr.get_or_create_feed(esdrDevice, device['latitude'], device['longitude'])
        feeds[awairDeviceId] = feed
    return feeds

In [10]:
def getEsdrFeedByDevice(awairDeviceId):
    esdrDevice = esdr.get_or_create_device(product, awairDeviceId)
    feed = esdr.get_or_create_feed(esdrDevice)
    return feed

In [11]:
def fetchAwairData(awairDeviceId,key,start,finish):
    url = '%sdevices/%s/events/15min-avg?from=%s&to=%s' % (awairRoot,awairDeviceId,start,finish)
    header = 'Authorization: Bearer %s' % key
    output = subprocess.check_output(["curl", "-q", url,"-H",header])
    data = json.loads(output)['data']
    return data

In [12]:
def isoTimestampToEpochTime(timestamp):
    date = datetime.datetime.strptime(timestamp,'%Y-%m-%dT%H:%M:%S.%fZ')
    epochTime = (date - datetime.datetime(1970, 1, 1)).total_seconds()
    return int(epochTime)
#isoTimestampToEpochTime('2016-08-28T23:30:00.000Z')

In [13]:
def formatAwairData(awairData):
    esdrData = {"channel_names":[],"data":[]}
    channels = awairData[0]['sensor'].keys()
    esdrData['channel_names'].extend(channels)
    for awairDatum in awairData:
        esdrDatum = []
        esdrDatum.append(isoTimestampToEpochTime(awairDatum['timestamp']))
        for prop in awairDatum['sensor']:
            esdrDatum.append(awairDatum['sensor'][prop])
        esdrData['data'].append(esdrDatum)
    return esdrData

In [14]:
def uploadDate(date):
    start = date.replace(hour=0,minute=0,second=0)
    finish = (start + datetime.timedelta(hours=24,minutes=5))
    print "fetching data from " + str(start) + " to " + str(finish)
    
    keys = parseForApiKeys('awair-sensors-metadata.csv')
    for key in keys:
        feeds = getEsdrFeeds(key)
        for feed in feeds:
            data = fetchAwairData(feed,key,start.isoformat(),finish.isoformat())
            if data:
                print "uploading %s data points to ESDR feed %s " % (len(data),feeds[feed]['id'])
                upload = formatAwairData(data)
                esdr.upload(feeds[feed], upload)
            else:
                print "no data for feed %s" % str(feeds[feed]['id'])
#date = datetime.datetime(2016, 12, 7)
#uploadDate(date)

In [15]:
def uploadDateForDevice(date, awairDeviceId, key):
    start = date.replace(hour=0,minute=0,second=0)
    finish = (start + datetime.timedelta(hours=24,minutes=5))
    print "fetching data from " + str(start) + " to " + str(finish) + " for device with Awair ID " + str(awairDeviceId)
    
    feed = getEsdrFeedByDevice(awairDeviceId)
    data = fetchAwairData(awairDeviceId,key,start.isoformat(),finish.isoformat())
    if data:
        print "uploading %s data points to ESDR feed %s " % (len(data),feed['id'])
        upload = formatAwairData(data)
        esdr.upload(feed, upload)
    else:
        print "no data for feed %s" % str(feed['id'])

In [37]:
def dailyUpload():
    #upload today
    today = datetime.datetime.utcnow()
    uploadDate(today)
dailyUpload()

fetching data from 2016-12-08 00:00:00.284000 to 2016-12-09 00:05:00.284000
uploading 91 data points to ESDR feed 7768 
uploading 91 data points to ESDR feed 7710 
uploading 91 data points to ESDR feed 7711 
uploading 91 data points to ESDR feed 7713 
uploading 91 data points to ESDR feed 7715 
no data for feed 7779


In [21]:
def uploadHistoricData(file, key):
    devices = parseForDevicesAndStartDates(file)
    for awairId in devices:
        startString = devices[awairId]
        start = datetime.datetime(int(startString[0:4]),int(startString[5:7]),int(startString[8:10]))
        for date in rrule.rrule(rrule.DAILY, start, until=datetime.date.today()):
            uploadDateForDevice(date,awairId, key)